In [36]:
import pandas as pd
import seaborn as sns

import sklearn
import json
import matplotlib.pyplot as plt
import time
import os
import random
import psutil
import re
from multiprocessing import Process

import math
import numpy as np
from sklearn import metrics
from subprocess import call
from numpy.random import seed
import sys
from scipy.stats import chi2_contingency
from scipy.stats import chi2 as chi_2
from sklearn.feature_selection import chi2

import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier as tratc
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

from sklearn import tree
from numpy import random
from multiprocessing import Process , Manager
import copy
from copy import deepcopy as cdc
import pandas as pd
import datetime 
from sklearn.linear_model import LogisticRegression
import psutil
from datetime import datetime  as dt
from xgboost import XGBClassifier
import xgboost
import datetime

In [37]:
sns.set(style="white",color_codes=True)

sys.setrecursionlimit(1000000)
seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED']='0'  
try :
        os.environ['OPENBLAS_NUM_THREADS']="1"
        import numpy
finally:
       if 'OPENBLAS_NUM_THREADS' in os.environ:
              del os.environ['OPENBLAS_NUM_THREADS']


In [38]:
def time_stamp():
     print(time.strftime("%Y-%m-%d-%H-%M",time.gmtime()))
    
def preprocessing():
    pass

In [39]:
    
class Assignment:

    
  def __init__(self,config_file_url):
    
    print('initialisation stage ')
    print(" data loading and sanity checks ")
    self.config_file_handle=open(config_file_url,"r",encoding='utf-8')
    print(config_file_url)
    
    assert self.config_file_handle, ' Could not open config File'
    
    self.config= json.load(self.config_file_handle)
 
    print('loading input dataset')
    
    self.input_data=pd.read_csv(self.config["DataURL"],keep_default_na=False,encoding='latin-1')  
    
    
    self.feature_data=self.input_data
    
    assert not self.input_data.empty, "Could not read input dataset/ input_data empty"
    
  def input_head(self):
    
       print(self.input_data[0:10])   
       print(self.input_data.shape)
 
    
  def input_distribution(self):  
      print('input data distribution')  
      for w in self.input_data:  
            print("\n\n")
            print(self.input_data[w].value_counts())
            
  def feature_distribution(self):
      print('target variable:approved')
      print('feature distribution with respect to Target variable: Approved')
      
      print(self.input_data['approved'].value_counts())
      
      print('Chi2 test on Target and Feature Distribution')
      Chi2_dict={}
      Chi2_list=[]
      for w in self.input_data:
        print("\n\n")
        print(" FEATURE :" + str(w))
        print(pd.crosstab(self.input_data[w],self.input_data.approved)) 
        
        stat,p,dof,expected=chi2_contingency(pd.crosstab(self.input_data[w],self.input_data.approved))
        
        prob = 0.95
        critical = chi_2.ppf(prob, dof)
        
        print('Chi2 test on Target and Feature Distribution')
      
        print('From chi2 Statistic probability=%.3f, critical=%.5f, stat=%.5f' % (prob, critical, stat))
        
        if abs(stat) >= critical:
            print('Dependent (reject H0)')
        else:
            print('Independent (fail to reject H0) ')
        
        # interpret p-value
        alpha = 1.0 - prob
        print('From Test significance=%.5f, p=%.5f' % (alpha, p))
        
        if p <= alpha:
            print('Dependent (reject H0)')
        else:
            print('Independent (fail to reject H0)')

        #self.input_data.plot(kind='scatter',y='',x='approved')
  
  def feature_engineering(self):
      listw=[]
      print(' Feature Engineering ')
        
      self.input_data['age']=None
      self.input_data['state']=None
      self.input_data['mail_accnt']=None
      self.input_data['house_type']=None
    
      for i,w in self.input_data.iterrows():
        
        w['age']=-1
        dateformat='%Y-%m-%d'
        dt1=time.strftime(dateformat,time.gmtime())    
        dt2=w['date_of_birth']
        dat1=datetime.datetime.strptime(dt1,dateformat)-datetime.datetime.strptime(dt2,dateformat)        
        w['age']=dat1.days    
        
        g= re.search(r'(\w\w)\s+\d{5}\s*$',w['address'])
                                                    
        if g:
            w['state']=list(g.groups('\1'))[0]
        else:
            w['state']=None
        
        mail=re.search(r'\w+@(\w+\.\w+)\s*',w['email'])
        
        if mail:
            w['mail_accnt']=list(mail.groups('\1'))[0]
        else:
            w['mail_accnt']=None
        
        apt=re.search(r'Apt',w['address'])

        suite=re.search(r'Suite',w['address'])
         
        if apt:
            w['house_type']='Apt'
                
        elif suite:
            w['house_type']='Studio'
        else:
            w['house_type']='others'
            
        listw.append(w)
      
      Feature_Engineered_Data=pd.DataFrame(listw,columns=listw[0].keys())
      
      self.input_data=Feature_Engineered_Data
      
      #self.input_data.to_csv('')
        
  # Pruning the High Cardinality Categorical Features  
  def feature_pruning(self):
    
    for w in self.input_data:
      
      if(self.input_data[w].dtype==object):  
        wc= self.input_data[w].value_counts()
        if len(wc)>4000:
            
            print('number of unique values in a categorical Data Column'+str(w)+ ' is '+str(len(wc))+' and it is high so pruning it')
            self.feature_data.drop([w],axis=1)
    
    print(self.feature_data[0:10])              
  
  # Finding the top AUC_PR from Grid Search Models
  def updating_o(self,model_perf,model_perf_):
  
   if 'AUC_PR' in model_perf_:
    o_auc_pr=model_perf_['AUC_PR']
    auc_pr=model_perf['AUC_PR']
    o_tp=model_perf['TP']
    tp=model_perf_['TP']

    if(  auc_pr > o_auc_pr ):

      for ky in model_perf.keys():

        model_perf_[ky]=model_perf[ky]

   if 'AUC_PR' not in model_perf_:

      for ky in model_perf.keys():

        model_perf_[ky]=model_perf[ky]
        """cdc( model_perf)
        """
   return model_perf_
  
  # ML modelling using grid search   
  def modelling(self,model,targt,targt2,prdictors,train, test ,model_typ,model_perf_list,model_perf_,model_props,ind=1,o_=0,idnt=''):
    '''print('hi')
    '''
    target=targt
    target2=targt2
    predictors=prdictors

    train=train
    test=test

    model.fit(train[predictors],train[target])

    model_predictions=model.predict(test[predictors])

    model_proba=model.predict_proba(test[predictors])

    test['predictions']=model_predictions

    test['probability']=model_proba[:,1]

    pcsv=test
    
    # Output the Predicted Probability if required 
    
    if(o_==1):
        pcsv['Probability_/|\\']=model_proba[:,1]
        pcsv['Predictions']=model_predictions
        pcsv.to_csv(r'Predictions'+idnt+'  ' +str( rand )+'.csv')
 
    tn,fp,fn,tp=confusion_matrix(test[target],model_predictions).ravel()
 
    prcision=tp/(1+tp+fp)
    rcall=tp/(tp+fn+1)
    f1=2*prcision*rcall/(prcision+rcall)
    auc_roc = metrics.roc_auc_score ( test [ target2 ] , model_proba[:,1] )
    auc_pr = metrics.average_precision_score ( test [ target2 ] , model_proba[:,1] )


    lng=len(pcsv)
    pcsv_Y=pcsv[ pcsv[ target2 ] ==1 ]
    pcsv_N=pcsv[ pcsv[ target2 ] ==0 ]

    model_perf={}
    model_perf['MODEL']=str(model_props)
    model_perf['TP']=tp
    model_perf['FP']=fp
    model_perf['TN']=tn
    model_perf['FN']=fn
    model_perf['PRECISION']=prcision
    model_perf['RECALL']=rcall
    model_perf['F1']=f1
    model_perf['AUC_ROC']=auc_roc
    model_perf['BER (balanced Error Rate)']=1-auc_roc
    model_perf['AUC_PR']=auc_pr
    model_perf['AVG Probability of Y']= str(sum(pcsv_Y['probability'])/len(pcsv_Y))
    model_perf['AVG Probability of N']=str(sum(pcsv_N['probability'])/len(pcsv_N))
    model_perf['Min Probability of Y']=str(min(pcsv_Y['probability']))
    model_perf['Max Probability of Y']=str(max(pcsv_Y['probability']))
    model_perf['Fcontrib']=''

    if(model_typ=='LR'):

       model_perf['Fcontrib']=str([ (predictors[i],model.coef_[0][i]) for i in range( len( predictors))]) + ' intercept: ' + str( model.intercept_)

    if(model_typ=='RF' or model_typ=='GBM'):

       model_perf['Fcontrib']=str( [ ( predictors[i],model.feature_importances_[i] )  for i in range( len( predictors ) ) ] )

    self.updating_o(model_perf,model_perf_)

    
    if(ind%10==0):
        
       print('currnt:'+'\n\n')
       self.printMtrics(model_perf)

       print(' top model '+'\n\n')
       self.printMtrics(model_perf_)

    pcsv={}
    train={}
    test={}
    pcsv_Y={}
    pcsv_N={}

    return auc_pr , model_perf

  # printing the metrics while on grid search 
  def printMtrics ( self,model_perf):

     model_props=model_perf['MODEL']
     o_tp=model_perf['TP']
     o_p_f=model_perf['FP']
     o_f_n=model_perf['FN']
     o_tn=model_perf['TN']
     o_prcision=model_perf['PRECISION']
     o_rcall = model_perf['RECALL']
     o_f1=model_perf['F1']
     o_auc_roc=model_perf['AUC_ROC']
     o_auc_pr=model_perf['AUC_PR']
     o_ber=model_perf['BER (balanced Error Rate)']
     o_Fcontrib=model_perf['Fcontrib']
     avg_Probability_F=model_perf['AVG Probability of Y']
     avg_Probability_N=model_perf['AVG Probability of N']
     min_Probability_F=model_perf['Min Probability of Y']
     max_Probability_F=model_perf['Max Probability of Y']

     print(' MODEL ' + str(model_props))
     print(' TP '+str(o_tp))
     print(' FP '+str(o_p_f))
     print(' FN '+str(o_f_n))
     print(' TN '+str(o_tn))
     print(' PRECISION ' +str(o_prcision))
     print(' RECALL '+str(o_rcall))
     print(' F1 '+str(o_f1))
     print(' auc roc ' + str(o_auc_roc))
     print(' auc pr '  + str(o_auc_pr))
     print(' Balanced Error Rate ' + str( 1 - o_auc_roc))
     
     print(' Features and contributions :' + str(o_Fcontrib))

     print('Avg Probability of Y : ' + str(avg_Probability_F))

     print('Avg Probability of N : ' + str(avg_Probability_N))

     print('Min Probability of Y : ' + str(min_Probability_F))

     print('Top Probability of Y : ' + str(max_Probability_F))

  #DECISION TREE GRID SEARCH

  def dtc(self,part_id):

    model_perf_list=[]
    model_perf_dict={}
    model_perf={}
    model_perf_={}

    cnt_model=0
    was=dt.now()

    j_=0

    pr=[]
    l1=0
    ind=0
    mkdepth_l= [16,14,12,10,9,7,6,4]
    minsamplesplit_l=[4,5,6,7,9,10,14,16]
    makftr_l=[0.2,0.5,0.7,'log2','auto','sqrt']
    minsamplelf_l=[4,5,6,8]
    minImpdcr_l=[0.01,0.001,0.02,0.002,0.005,0.05]
    criteria_l=['gini','entropy']

    num_iters=len(mkdepth_l)*len(minsamplesplit_l)*len(makftr_l)*len(minsamplelf_l) * len(minImpdcr_l) * len(criteria_l)

    i_=0
    for makdepth in mkdepth_l:
       for minsamplesplit in minsamplesplit_l:
         for makftr in makftr_l:
            for minsamplelf in minsamplelf_l:
              for criteria in criteria_l:
                for minImpdcr in minImpdcr_l:

                 st= int(num_iters /(psutil.cpu_count()-2)) * part_id
                 lst=int(num_iters /(psutil.cpu_count()-2)) * ( part_id+1 )

                 if i_>=st and i_ <lst:

                  cnt_model=cnt_model+1
                  last_ = was
                  was = dt.now()

                  ind=ind+1

                  print ( str(i_) + ' / '+str(num_iters)+  ' duration for modelling : '+str(was-last_)+ ' till is : '+str(dt.now()-start)+ '  DT :' + ' minsamplelf  '+str(minsamplelf)+ ' maksFitursslction : '
                         +str(makftr) + ' maksDpth : ' +str(makdepth) + ' minsamplesplit : ' + str(minsamplesplit) + 'min Impurity dcrs' +str(minImpdcr))

                  model_props={}
                  model_props['algo']='DT'
                  model_props['Maks Fiturs']=makftr
                  model_props['criteria']=criteria
                  model_props['Min Sample leaf']=minsamplelf
                  model_props['Maks depth']=makdepth
                  model_props['Min sampleing split']=minsamplesplit
                  model_props['class weight']='balanced'
                  model_props['Min Impurity decrease ']=minImpdcr

                  dtr_wt=DTC( max_depth=makdepth,min_samples_split=minsamplesplit,criterion=criteria ,max_features=makftr, class_weight='balanced', min_samples_leaf=minsamplelf , min_impurity_decrease=minImpdcr)

                  model_props1={}
                  model_props1['algo']='DT'
                  model_props1['Maks Fiturs']=makftr
                  model_props1['criteria']=criteria
                  model_props1['Min Sample leaf']=minsamplelf
                  model_props1['Maks depth']=makdepth
                  model_props1['Min sampleing split']=minsamplesplit
                  model_props1['class weight']='None'
                  model_props1[' Min Impurity decrease ']=minImpdcr

                  dtr = DTC( max_depth=makdepth,min_samples_split=minsamplesplit,criterion=criteria, max_features=makftr, min_samples_leaf=minsamplelf , min_impurity_decrease=minImpdcr)

                  auc_pr,model_perf  =self.modelling(dtr_wt,target,target2,predictors,train,test,'DT',model_perf_list,model_perf_,model_props,ind)

                  ind=ind+1

                  auc_pr,model_perf_n  =self.modelling(dtr,target,target2,predictors,train,test,'DT',model_perf_list,model_perf_,model_props1,ind)

                  model_perf_list.append(model_perf)

                  model_perf_list.append(model_perf_n)

                 i_=i_+1

                 if(i_%1000==0):
                               
                     model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())
                     model_perf_ranking.to_csv(r'model_accuracy_dt_'+'part_'+str(part_id)+'_'+str( rand ) +' .csv')
                     model_perf_ranking={}
           
    model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())
    model_perf_ranking.to_csv(r'model_accuracy_dt_'+'part_'+str(part_id)+'_'+str( rand ) +' .csv')

    model_perf_ranking={}
    
  # GRID SEARCH ON RANDOM FOREST
  def rf(self,part_id):
    
    njobs=psutil.cpu_count()-1
    
    model_perf_list =[]
    model_perf ={}
    model_perf_={}

    cnt_model=0
    was=dt.now()
    j_=0
    ind=0

    makdepth_l=[16,12,8,4]
    makftr_l=[0.2,0.5,'log2','sqrt','auto']
    makiter_l=[500,700,1200]
    minsamplesplit_l=[5,9,14,16]
    minsamplelf_l=[4,6,7]
    minImpdcr_l=[ 0.01, 0.001,0.02,0.002,0.05,0.005]

    criteria_l=['gini','entropy']
    i_=0

    num_iters=len(makdepth_l)*len(makftr_l) * len(makiter_l) * len(minsamplesplit_l)  * len(minsamplelf_l) * len(criteria_l)

    print( ' RF iters ' +str(num_iters))

    for makdepth in makdepth_l:
      for makftr in makftr_l:
        for makiter in makiter_l:
          for minsamplesplit in minsamplesplit_l:
            for minsamplelf in minsamplelf_l:
              for criteria in criteria_l:
                  st=int(num_iters/(psutil.cpu_count()-2 ))*part_id
                  last=int(num_iters/(psutil.cpu_count()-2))*(1+part_id)
                  """print('st:'+str(st)+ 'lst:' +str(last))
                  """

                  if((i_ >=  int( num_iters /(psutil.cpu_count()-2)) * part_id ) and ( i_ < int(num_iters/(psutil.cpu_count()-2)) *(1+part_id) ) ):

                      print('st:'+str(st)+ ' lst:' + str(last) + 'part :' +str(part_id))

                      last_=was
                      was=dt.now()
                      ind=ind+1

                      print( str(cnt_model) + '/'+ str(num_iters)+ '  ' + 'duration for modelling: '+ str( was-last_ ) + 'till is '+str(dt.now()-start)+' RF :' + ' makftr : ' +str(makftr) + ' makiter : ' +str(makiter) + ' minsamplesplit : ' +str(minsamplesplit) + '  makdepth : ' +str(makdepth) + ' minsamplelf :' +str(minsamplelf))

                      cnt_model=cnt_model+1

                      model_props={}
                      model_props['algo']='RF'
                      model_props['Maks Fiturs']=makftr
                      model_props['Maks depth']=makdepth
                      model_props['Min Sampling split']=minsamplesplit
                      model_props['Min Sampling leaf']=minsamplelf
                      model_props['Criteria']=criteria

                      rft=rfc( max_depth=makdepth ,  n_jobs=njobs , min_samples_leaf=minsamplelf, max_features=makftr, n_estimators=makiter, criterion=criteria, class_weight='balanced')

                      auc_pr,model_perf=self.modelling(rft , target, target2, predictors, train ,   test, 'RF' , model_perf_list , model_perf_ , model_props, ind)

                      model_perf_list.append(model_perf)

                  i_ = i_ +1

                  if(i_%1000==0):
                               
                     model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())
                     model_perf_ranking.to_csv(r'model_accuracy_rf_'+'part_'+str(part_id)+'_'+str( rand ) +' .csv')
                     model_perf_ranking={}
           

    model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())

    model_perf_ranking.to_csv(r'model_accuracy_rf_'+'part_'+str(part_id)+'_'+str( rand )+' .csv')

    model_perf_ranking={}

  #GRID SEARCH On GBM

  def gbm(self,part_id):

    njobs=psutil.cpu_count()-1

    model_perf_list =[]
    model_perf = {}
    model_perf_  = {}

    j_=0
    i_=0

    cnt_model=0
    was=dt.now()
    ind=0
    makdepth_l= [16,12,8,4]
    los_l= ['deviance','exponential']
    subsample_l=[0.1,0.4,0.9]
    makftr_l=[0.2,0.5,'log2','sqrt','auto']
    makiter_l=[100,500,700,1000,1200]
    lurningrayt_l=[0.1,0.001,0.01,0.2,0.002,0.02,0.06,0.05,0.005]
    minsamplesplit_l=[4,6,8,10,16]

    num_iters=len(makdepth_l) * len(los_l) * len(subsample_l) * len(makftr_l) * len(makiter_l) * len(lurningrayt_l) * len( minsamplesplit_l)

    for makdepth in makdepth_l:
      for los in los_l:
        for subsample in  subsample_l:
          for makftr in makftr_l:
            for makiter in makiter_l:
              for lurningrayt in lurningrayt_l:
                for minsamplesplit in minsamplesplit_l:

                 st= num_iters/(psutil.cpu_count()-2)*part_id
                 lst=(num_iters/(psutil.cpu_count()-2))*(1+part_id)

                 if ( ( i_ >= st ) and ( i_ < lst ) ) :
                  print('st :' +str(st) + ' lst :' + str(lst) )
                  cnt_model=cnt_model+1
                  last_=was
                  was=dt.now()

                  ind=ind+1

                  print(str(cnt_model) + ' / '+  str(num_iters)  +' duration of modelling : ' + str(was-last_) + ' till is : '+str(dt.now()-start)+ '  GBM :' + ' loss: ' +str(los) + ' Subsample :'+str(subsample) + ' maksFiturslction : ' +str(makftr) + ' makiter: '+ str(makiter) + ' lurningrayt : ' + str(lurningrayt) + ' maksDpth : ' + str(makdepth) + ' minsamplesplit : ' + str(minsamplesplit))

                 model_props={}
                 model_props['algo']='GBM'
                 model_props['loss']=los
                 model_props['Subsampleing']=subsample
                 model_props['NumTrs']=makiter
                 model_props['Maks Fiturs']=makftr
                 model_props['Learning rate']=lurningrayt
                 model_props['Maks depth']=makdepth
                 model_props['Min sampleing split']=minsamplesplit

                 gbt=gbc(loss=los,subsample=subsample,n_estimators=makiter,min_samples_split=minsamplesplit,max_depth=makdepth,learning_rate=lurningrayt )

                 auc_pr,model_perf  =self.modelling(gbt,target,target2,predictors,train,test,'GBM', model_perf_list, model_perf_,model_props,ind )

                 model_perf_list.append(model_perf)

                 i_=i_+1

                 if(i_%1000==0):
                               
                     model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())
                     model_perf_ranking.to_csv(r'model_accuracy_gbm_'+'part_'+str(part_id)+'_'+str( rand ) +' .csv')
                     model_perf_ranking={}
           
    model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())
    model_perf_ranking.to_csv(r'model_accuracy_gbm_'+'part_'+str(part_id)+'_'+str( rand ) +' .csv')
    print(model_perf_ranking[0:10])
    model_perf_ranking={}
   
  # GRID SEARCH ON XGBOOST
  def gboost(self,part_id):

    njobs=psutil.cpu_count()-1

    model_perf_list=[]
    model_perf={}
    model_perf_ = {}

    j_=0
    i_=0

    cnt_model=0
    was=dt.now()
    ind=0
    makdepth_l=[16,12]
    los_l=['binary:logistic']
    subsample_l=[0.1,0.4]
    imb_l=[0.001,0.01,0.002]
    makftr_l=[0.1,0.5]
    makiter_l=[1200]
    iters_l=[1000,2000]
    lurningrayt_l=[0.001,0.01,0.02,0.002,0.05,0.005]
    minsamplesplit_l=[4]
    gamma_l=[0,5]
    makdtstp_l=[0.01,0.05,0.1]

    boost_l=['gbtree','dart','gblinear']
    colsamplebt_l=[0.5]
    rgalp_l=[0.5]
    rglmbd_l=[0.5]

    num_iters=len(makdepth_l) * len(los_l)  * len(imb_l) *len(iters_l) *len(rglmbd_l)* len(makftr_l) * len(makiter_l) * len(subsample_l) *  len(minsamplesplit_l)  * len ( gamma_l)  * len ( lurningrayt_l)  * len(rgalp_l) *len(colsamplebt_l)  * len (makdtstp_l) * len(boost_l) * len(rglmbd_l)

    for colsamplebt in colsamplebt_l:
     for makdepth in makdepth_l:
      for subsample in subsample_l:
        for makftr in makftr_l:
         for makiter in makiter_l:
          for lurningrayt in lurningrayt_l:
            for imb in imb_l:
              for iters in iters_l:
               for gamma in gamma_l:
                for minsamplesplit in minsamplesplit_l:
                  for makdtstp in makdtstp_l:
                    for boost in boost_l:
                      for lmbd in rglmbd_l:
                       for alpha in rgalp_l:
                           
                        st=num_iters/(psutil.cpu_count()-2)*part_id
                        lst=(num_iters/(psutil.cpu_count()-2))* ( 1+ part_id )

                        if ( ( i_ >=st ) and ( i_ <lst ) ):

                           print('st :'+str(st) + ' lst :' + str(lst))
                           cnt_model=cnt_model+1
                           last_=was
                           was=dt.now()
                           ind=ind+1
                           print(str(cnt_model) + ' / ' + str(num_iters) + ' duration of modelling : '+ str(dt.now() -start)+ ' GBoost: ' + '  Subsample :' + str(subsample) + 'maksFiturslction : ' + str(makftr) +'makiter : '+ str(makiter) +'lurningrayt : '+str(lurningrayt) +'maksDpth:'+ str(makdepth) + 'minsamplesplit:' + str(minsamplesplit) + 'gamma :' + str(gamma) + 'alpha :' + str(alpha) + 'imbalancd wt:' + str(imb))
                           model_props={}
                           model_props['algo']='GBOOST'
                           model_props['Subsampleing']=subsample

                           model_props['NumTrs']=makiter
                           model_props['Maks Fiturs']=makftr
                           model_props['Learning rate']=lurningrayt
                           model_props['Maks depth']=makdepth
                           model_props['Min sampleing split']=minsamplesplit
                           model_props['gamma']=gamma
                           model_props['alpha']=rgalp_l
                           model_props['imb']=imb
                           model_props['boostr']=boost
                           model_props['colsamplebyTr']=colsamplebt
                           model_props['maks dlta stp']=makdtstp

                           gbst =  XGBClassifier( max_depth=makdepth, objective='binary:logistic' , scale_pos_weight=imb ,  booster=boost , learning_rate=lurningrayt , min_child_weight=makftr , colsamplebytree=colsamplebt , reg_alpha=alpha , gamma= gamma , reg_lambda=lmbd )

                           auc_pr,model_perf=self.modelling(gbst, target , target2 , predictors , train ,test , 'GBoost' , model_perf_list,model_perf_, model_props,ind)

                           model_perf_list.append(model_perf)

                        i_ = i_ +1
                        
                        if(i_%1000==0):

                              model_perf_ranking=pd.DataFrame(model_perf_list,columns=model_perf.keys())

                              model_perf_ranking.to_csv(r'model_accuracy_gboost_'+'part_'+str(part_id)+'_'+str( rand )+' .csv')
                              
                              model_perf_ranking={}
  

 

            Feature Importance From Random Forest
            
             
	  'relationship'              0.15417    
      'capital_gain'              0.14262     
      'marital_status'            0.12675     
      'education_num'             0.10298    
      'age'                       0.08588     
      'date_of_birth'             0.06169     
      'hours_per_week'            0.04874     
      'education_level'           0.03426     
      'occupation'                0.03330     
      'inquiry_purpose_code'      0.03099     
      'address'                   0.02673  
      'email'                     0.02618    
      'capital_loss'              0.02380     
      'state'                     0.02232     
      'gender'                    0.01786     
      'mail_accnt'                0.01724     
      'workclass'                 0.01024    
      'account_type'              0.00976     
      'institute_type'            0.00703     
      'house_type'                0.00694    
      'asset_class_cd'            0.00504   
      'portfolio_type'            0.00488    
      'asset_code'                0.00050   

      

In [40]:
if __name__=='__main__':
  assignment=Assignment("C:/Users/lenovo/Documents/Assignment 1/config.json")
  assignment.input_head()

  #assignment.input_distribution()

  #assignment.input_data[0:10]

  assignment.feature_engineering()
  assignment.feature_distribution()
  
  #assignment.feature_pruning()
   
  
  pcsv_full=assignment.input_data
   
  pcsv_full[pcsv_full==np.inf]=np.nan
  pcsv_full.fillna(0,inplace=True)
  
  target='approved'

  target2='alt_flag'

  pcsv_full[target2]=[0 if a==0 else 1 for a in pcsv_full[target]]

  predictors=[a for a in pcsv_full if a not in [target,target2]]

  # Converting string/categorical variables to indices
    
  pcsv2=pd.DataFrame({col: pcsv_full[col].astype('category').cat.codes for col in pcsv_full}, index=pcsv_full.index)
   
  #80-20% Split of Train and Test Data

  positives=pcsv2[pcsv2[target]==1]

  negatives=pcsv2[pcsv2[target]==0]

  positives1=positives[round(len(positives)/5):]
  positives2=positives[:round(len(positives)/5)]
  Positives_80=positives1
  Positives_20=positives2

  train=pd.concat([Positives_80,negatives[round(len(negatives)/5):]])
  test= pd.concat([Positives_20,negatives[:round(len(negatives)/5)]])

  trtst=pcsv2
  print(" TRAIN DATA TARGET DISTRIBUTION ")
  print(train[target].value_counts())

  tr2=train
  tst2= test
  print(" TEST DATA TARGET DISTRIBUTION ")
  print( test[target2].value_counts() )
  train[target]=tr2[target]
  test[target]=tst2[target]

  #train=pcsv2
  #test=train
  ui_=0
  pr=[]
  # Dividing workload for every processor cores using multi-core programming
  for part_id in range(psutil.cpu_count()-2):

    if ui_==0:
        #pass
        #assignment.rf(part_id)
        
        assignment.dtc(part_id)
        
        #assignment.gboost(part_id)
        
    if ui_!=0:

      arg =[part_id]

      p= Process( target=assignment.dtc , args =(arg) )

      p.start()

      pr.append(p)

        
        
  if ui_!=0:

   # stalling for all cores processes to complete

   for wg in pr:
     wg.join()

  algo='rf'
  rst_lst=[]
   
  print(' dt ')
  
  # TO PROCURE PREDICTED PROBABILITY
  #dtr = DTC( max_depth=16,min_samples_split=6,criterion='entropy', max_features=0.5, min_samples_leaf=4 , min_impurity_decrease=0.001)
  #auc_pr,model_perf  =modelling(dtr,target,target2,predictors,train,train,'DT',pr,model_perf,{},ind=1,o_=1)
   
  """  GRID SEARCH on Other Techniques such as RF and GBoost Takes more time from PC.. needs GPUs """

initialisation stage 
 data loading and sanity checks 
C:/Users/lenovo/Documents/Assignment 1/config.json
loading input dataset
                                user_id   gender date_of_birth  \
0  307c73a3-0c67-43b5-b0d1-62ad5f00b52a   Female    1987-01-03   
1  519481f1-604d-48b6-aa47-4123a982431a     Male    1970-01-07   
2  f66fe2c6-53e3-46ce-8321-e530b4af1ace     Male    1973-01-06   
3  91360ed7-72d1-4f6e-b492-d71ef4480061   Female    2001-12-30   
4  0e1977f3-3bef-4283-8d0a-d60b0f6ff480     Male    1986-01-03   
5  3c8fbbaf-cd9d-4e17-8cd4-fc7c01eb9203   Female    1956-01-11   
6  27256e97-6bae-4ce6-9b80-58fb66a3d452   Female    2001-12-30   
7  745b9022-fe0e-4d8e-8056-96f56613260f   Female    1996-12-31   
8  9b56df25-024a-47fe-b895-70317c7b63ce     Male    1977-01-05   
9  3257eeda-137c-4d8c-a295-b497a3211a48   Female    1963-01-09   

           workclass education_level  education_num       marital_status  \
0            Private            11th            7.0        Never-marr

KeyboardInterrupt: 

In [ ]:
""" BEST MODEL SO FAR FROM GRID SEARCH Decision Tree IS 


     MODEL	
     {'algo': 'DT', 'Maks Fiturs': 0.5, 'criteria': 'gini', 'Min Sample leaf': 5, 'Maks depth': 6,
     'Min sampling split': 16, 'class weight': 'None', ' Min Impurity decrease ': 0.002}
     
     TP	       : 1575
     FP	       :345	 
     TN	       :6458	 
     FN	       :667	 
     PRECISION : 0.819885476	 
     RECALL    : 0.702184574	 
     F1	       : 0.228269208	 
     AUC_ROC   : 0.844584164	
     BER (balanced Error Rate) : 0.155415836	 
     AUC_PR	   : 0.746829475	
     AVG Probability of Y	0.581622991	 
     AVG Probability of N	0.03553647
     Min Probability of Y: 0
     Max Probability of Y: 1


BEST MODEL FROM GBM GRID SEARCH IS:
     
     MODEL {'algo': 'GBM', 'loss': 'deviance', 'Subsampling': 0.1, 'NumTrs': 500, 'Maks Fiturs': 0.2, 
     'Learning rate': 0.01, 'Maks depth': 16, 'Min sampling split': 16}
 
     TP 1219
     FP 351
     FN 1023
     TN 6452
     PRECISION 0.7759388924252069
     RECALL 0.543468568880963
     F1 0.1818129873428656
     auc roc 0.9053952820048562
     auc pr 0.7686359862739767
     Balanced Error Rate 0.0946047179951438
     Features and contributions :
     [('gender', 0.0029153905578678005), ('date_of_birth', 0.047829859206671994), ('workclass', 0.030329925958473215),
     ('education_level', 0.019853006561685118), ('education_num', 0.07527902231522515), 
     ('marital_status', 0.013478993158479344), ('occupation', 0.04370178221814543),
     ('relationship', 0.09103391973722465), ('capital_gain', 0.09553318766971383),
     ('capital_loss', 0.026982921494765805), ('hours_per_week', 0.06418012921459622), 
     ('address', 0.10478250232306009), ('email', 0.10669845453478409), 
     ('inquiry_purpose_code', 0.03558648819881797), ('institute_type', 0.013275798504502032),
     ('account_type', 0.021349947358194114), ('asset_class_cd', 0.007336848507644418), 
     ('asset_code', 0.006365574874894791), ('portfolio_type', 0.004794805620225104),
     ('age', 0.06459150864450301), ('state', 0.06716462612907452), ('mail_accnt', 0.045280960981271344), 
     ('house_type', 0.011654346230179923)]
     
     Avg Probability of Y : 0.5250741707918904
     Avg Probability of N : 0.10669522865578085
     Min Probability of Y : 0.009516564771497529
     Top Probability of Y : 0.9768237363604723


    IT has the Highest AUC_PR which is the metric for classs imbalanced dataset



This model shows answers a question of whether to approve or not approve a credit card application 
for an applicant

"""

2.how much to approve for an applicant depends upon his credit history and credit score 
    which is based on:
        
        a. how consistent he has payed off his loans/bills (35%)
        b. current loan/bills he has to pay  (30%)
        c. new credit (10%)
        d. type of credit (10%)
        f, For how long he has been consistently paying off his bills/loans (15%)  
        g. unused credits adds to the score
        
        From Salary:
        We can decide upon the amount of credit to an applicant based on their salary statements:
        with an EMI Ranging upto 65% of Salary and loan amount not more than a years salary or 
        $8000 whichever is lower
        
        From CIBIL Score or Credit Score:
        Credit Score (CIBIL Score) Ranges and cap Amount we can approve for an applicant:
                800-850         upto $8000
                740-799         upto $7000
                670-739         upto $6000
                580-669         upto $5000
        
        From Collateral/assets:
          For a new guy whose scores are not known, upto 70% of amount on the valuation of 
          collateral/assets can be approved on the credit card. 
          But this is an 1 hr evaluation of credit so this may not be viable option
        
        From Amount of Tax He is paying to Govt or he is yet to pay should be considered  
        
        From his partner/Parents/Guardian/Gauranteer Salary or CIBIL Score or Collateral 
        or Tax paid
        
        
        Or a Weighted combination of above using a Polynomial Regression can be obtained 
        to provide an amount to to approve for an applicant 
        